In [1]:
from google.colab import drive ; drive.mount("/content/drive")

Mounted at /content/drive


In [167]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import re
import datetime


print(f"Numpy Version {np.__version__}")
print(f"Pandas Version {pd.__version__}")
print(f"Tensorflow Version {tf.__version__}")

Numpy Version 1.18.5
Pandas Version 1.1.3
Tensorflow Version 2.3.0


In [216]:
path = "/content/drive/My Drive/Sales_Data"
files = [file for file in os.listdir(path)if not file.startswith('.')]

FullData = pd.DataFrame()

for file in files:
  datamonth = pd.read_csv(path + "/" + file)
  FullData = pd.concat([FullData,datamonth],axis = 0)

FullData.head(10)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,141234,iPhone,1,700,01/22/19 21:25,"944 Walnut St, Boston, MA 02215"
1,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,"185 Maple St, Portland, OR 97035"
2,141236,Wired Headphones,2,11.99,01/17/19 13:33,"538 Adams St, San Francisco, CA 94016"
3,141237,27in FHD Monitor,1,149.99,01/05/19 20:33,"738 10th St, Los Angeles, CA 90001"
4,141238,Wired Headphones,1,11.99,01/25/19 11:59,"387 10th St, Austin, TX 73301"
5,141239,AAA Batteries (4-pack),1,2.99,01/29/19 20:22,"775 Willow St, San Francisco, CA 94016"
6,141240,27in 4K Gaming Monitor,1,389.99,01/26/19 12:16,"979 Park St, Los Angeles, CA 90001"
7,141241,USB-C Charging Cable,1,11.95,01/05/19 12:04,"181 6th St, San Francisco, CA 94016"
8,141242,Bose SoundSport Headphones,1,99.99,01/01/19 10:30,"867 Willow St, Los Angeles, CA 90001"
9,141243,Apple Airpods Headphones,1,150,01/22/19 21:20,"657 Johnson St, San Francisco, CA 94016"


In [217]:
FullData.to_csv("Fulldata.csv",index=False)

In [218]:
df = FullData.copy()

In [219]:
print(f"row {df.shape[0]} , Columns {df.shape[1]}" )
print(f"Contain Nans ? {df.isna().sum()}")


row 186850 , Columns 6
Contain Nans ? Order ID            545
Product             545
Quantity Ordered    545
Price Each          545
Order Date          545
Purchase Address    545
dtype: int64


In [220]:
df.dropna(how="all" , inplace=True)
print(f"Contain Nans ? {df.isna().sum()}")

Contain Nans ? Order ID            0
Product             0
Quantity Ordered    0
Price Each          0
Order Date          0
Purchase Address    0
dtype: int64


In [229]:
satanIndex = df[df["Purchase Address"] == "Purchase Address"].index
dfCleaned = df.drop(index=[satanIndex][0])
dfCleaned[dfCleaned["Purchase Address"] == "Purcgase Address"]

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address


In [232]:
# Adding More columns
def get_city(text):
  return text.split(",")[1].strip(" ")
def get_state(text):
  return text.split(",")[2].split(" ")[1].strip(" ")

dfCleaned["city"] = dfCleaned["Purchase Address"].apply(lambda text: f"{get_city(text)}({get_state(text)})")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [233]:
dfCleaned = dfCleaned.drop(["Purchase Address"],axis = 1)
dfCleaned

,Order ID,Product,Quantity Ordered,Price Each,Order Date,city
0,141234,iPhone,1,700,01/22/19 21:25,Boston(MA)
1,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,Portland(OR)
2,141236,Wired Headphones,2,11.99,01/17/19 13:33,San Francisco(CA)
3,141237,27in FHD Monitor,1,149.99,01/05/19 20:33,Los Angeles(CA)
4,141238,Wired Headphones,1,11.99,01/25/19 11:59,Austin(TX)
...,...,...,...,...,...,...
15221,176554,Lightning Charging Cable,1,14.95,03/24/19 11:14,San Francisco(CA)
15222,176555,27in FHD Monitor,1,149.99,03/22/19 20:27,San Francisco(CA)
15223,176556,AAA Batteries (4-pack),3,2.99,03/14/19 10:29,Los Angeles(CA)
15224,176557,iPhone,1,700,03/30/19 12:32,San Francisco(CA)


In [238]:
# Grouping Data
dfCleaned["ordertime"] = dfCleaned["Order Date"].str[-5:]
dfCleaned["orderdate"] = dfCleaned["Order Date"].str[:8]
dfCleaned["month"] = dfCleaned["orderdate"].str[0:2]
dfCleaned.drop(["Order Date"],axis = 1, inplace = True)



In [225]:
def ConvertToDate(dateee):
  dateee = str(dateee)
  dateee = re.sub("/","-",dateee)
  return dateee


ConvertToDate("01/02/03") 



'01-02-03'

In [239]:
dfCleaned["day"] = dfCleaned["orderdate"].str[3:5]
dfCleaned["orderdate"] = dfCleaned["orderdate"].apply(ConvertToDate)
dfCleaned["orderdate"] = pd.to_datetime(dfCleaned["orderdate"])
dfCleaned["isStartMonth"] = dfCleaned["orderdate"].dt.is_month_start
dfCleaned["isEndMonth"] = dfCleaned["orderdate"].dt.is_month_end

In [240]:
dfCleaned["Sales"] = dfCleaned["Quantity Ordered"].astype("int") * dfCleaned["Price Each"].astype("float")
dfCleaned["Sales"]

0        700.00
1         14.95
2         23.98
3        149.99
4         11.99
          ...  
15221     14.95
15222    149.99
15223      8.97
15224    700.00
15225     14.95
Name: Sales, Length: 182735, dtype: float64

In [257]:
dfCleaned.columns = ["orderid","product","quantity_ordered","price_each","city","ordertime","orderdate","month","day","isStartMonth","isEndMonth","Sales"]

In [262]:
dfCleaned.quantity_ordered = dfCleaned["quantity_ordered"].astype("int")
dfCleaned.price_each = dfCleaned["price_each"].astype("float")

In [266]:
dfCleaned.groupby(["month"]).sum()

,quantity_ordered,price_each,isStartMonth,isEndMonth,Sales
month,,,,,
01,10670,1.776156e+06,330,271,1.786511e+06
02,13162,2.145209e+06,405,397,2.158127e+06
03,16697,2.740393e+06,460,459,2.755969e+06
04,20226,3.313522e+06,596,617,3.336376e+06
05,18344,3.084756e+06,509,465,3.101881e+06
06,14964,2.508863e+06,447,437,2.524465e+06
07,15781,2.572765e+06,469,475,2.587445e+06
08,13185,2.178184e+06,364,412,2.191698e+06
09,12827,2.038020e+06,372,373,2.050361e+06


In [263]:
dfCleaned.groupby(["month"]).aggregate({np.sum,np.mean,np.median})["Sales"]

,median,mean,sum
month,,,
01,14.95,188.053820,1.786511e+06
02,14.95,184.046348,2.158127e+06
03,14.95,185.113474,2.755969e+06
04,14.95,185.519151,3.336376e+06
05,14.95,190.486431,3.101881e+06
06,14.95,189.923638,2.524465e+06
07,14.95,184.396017,2.587445e+06
08,14.95,187.020933,2.191698e+06
09,14.95,180.251539,2.050361e+06


In [251]:
dfCleaned.groupby(["month"]).aggregate({max,min})["Sales"]

,min,max
month,,
01,2.99,1999.98
02,2.99,1700.00
03,2.99,1700.00
04,2.99,3400.00
05,2.99,3400.00
06,2.99,3400.00
07,2.99,1700.00
08,2.99,1700.00
09,2.99,1700.00


In [268]:
len(dfCleaned.groupby(["day"]).head(1)) 

31

In [270]:
dfCleaned.groupby(['day']).aggregate({np.sum,np.mean,np.median})[["quantity_ordered","Sales"]]

quantity_ordered                  Sales                        
              median      mean   sum median        mean         sum
day                                                                
01                 1  1.135721  6753  14.95  192.394484  1143977.60
02                 1  1.123767  6837  14.95  182.864065  1112544.97
03                 1  1.123690  6432  14.95  186.145273  1065495.54
04                 1  1.121767  6679  14.95  191.627388  1140949.47
05                 1  1.131535  6710  14.95  187.948926  1114537.13
06                 1  1.112224  6660  14.95  189.239641  1133166.97
07                 1  1.122256  6646  14.95  180.739772  1070340.93
08                 1  1.123513  6704  14.95  181.785460  1084713.84
09                 1  1.117676  6715  14.95  190.695498  1145698.55
10                 1  1.114332  6842  14.95  187.025143  1148334.38
11                 1  1.113659  6653  14.95  193.086917  1153501.24
12                 1  1.137839  6802  14.95  181.639304  1085839.76
13                 1  1.128267  6993  14.95  179.608283  1113212.14
14                 1  1.128214  6802  14.95  184.030514  1109519.97
15                 1  1.111704  6668  14.95  182.787701  1096360.63
16                 1  1.127936  6771  14.95  179.500120  1077539.22
17                 1  1.128197  6838  14.95  184.100655  1115834.07
18                 1  1.113817  6723  14.95  188.838072  1139826.60
19                 1  1.131532  6908  14.95  177.232329  1082003.37
20                 1  1.126404  6817  14.95  186.269739  1127304.46
21                 1  1.127093  6731  14.95  185.054156  1105143.42
22                 1  1.127440  6759  14.95  185.859011  1114224.77
23                 1  1.126768  6693  14.95  179.803976  1068035.62
24                 1  1.123260  6780  14.95  183.190557  1105738.20
25                 1  1.122569  6695  14.95  192.876590  1150315.98
26                 1  1.140241  6911  14.95  185.544265  1124583.79
27                 1  1.116531  6707  14.95  184.806972  1110135.48
28                 1  1.125021  6749  14.95  183.086963  1098338.69
29                 1  1.122315  6322  14.95  190.105931  1070866.71
30                 1  1.121837  6206  14.95  184.891746  1022821.14
31                 1  1.128608  3949  14.95  185.445879   648875.13

In [250]:
dfCleaned.groupby(["month"]).head(1)

,Order ID,Product,Quantity Ordered,Price Each,city,ordertime,orderdate,month,day,isStartMonth,isEndMonth,Sales
0,141234,iPhone,1,700,Boston(MA),21:25,2019-01-22,01,22,False,False,700.00
4199,145212,Lightning Charging Cable,1,14.95,Dallas(TX),00:36,2019-02-01,02,01,True,False,14.95
0,194095,Wired Headphones,1,11.99,New York City(NY),17:14,2019-05-16,05,16,False,False,11.99
1750,195748,27in FHD Monitor,1,149.99,San Francisco(CA),03:26,2019-06-01,06,01,True,False,149.99
0,222910,Apple Airpods Headphones,1,150,Atlanta(GA),16:51,2019-07-26,07,26,False,False,150.00
2744,225541,27in 4K Gaming Monitor,1,389.99,Los Angeles(CA),01:15,2019-08-01,08,01,True,False,389.99
2251,238834,Apple Airpods Headphones,1,150,San Francisco(CA),04:13,2019-09-01,09,01,True,False,150.00
0,259358,34in Ultrawide Monitor,1,379.99,Dallas(TX),10:56,2019-10-28,10,28,False,False,379.99
1858,261127,34in Ultrawide Monitor,1,379.99,Seattle(WA),01:49,2019-11-01,11,01,True,False,379.99
0,295665,Macbook Pro Laptop,1,1700,New York City(NY),00:01,2019-12-30,12,30,False,False,1700.00
